In [1]:
import os

In [2]:
%pwd

'c:\\Data-Science\\ML Projects\\End_to_End_ML_Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Data-Science\\ML Projects\\End_to_End_ML_Project'

### Entity


In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

### Configuration Manager


In [6]:
from src.e2emlproject.constants import *
from src.e2emlproject.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_file_path=CONFIG_FILE_PATH,
        params_file_path=PARAMS_FILE_PATH,
        schema_file_path=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifcats_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config

## Components


In [7]:
import os
import urllib.request as request
import zipfile
from src.e2emlproject.logging import logger
from src.e2emlproject.utils.common import get_size


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL, filename=self.config.local_data_file
            )
            logger.info(
                f"Downloaded {filename} of size {get_size(Path(self.config.local_data_file))} from {self.config.source_URL} with the following info: \n {headers}!!"
            )
        else:
            logger.warning(
                f"File already exists of size: {get_size(Path(self.config.local_data_file))}."
            )

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        logger.info("Zip extration started!")
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info("Zip extraction completed!")

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    print("Data Ingestion Config:")
    data_ingesttion = DataIngestion(config=data_ingestion_config)
    data_ingesttion.download_file()
    data_ingesttion.extract_zip_file()
    logger.info("Data Ingestion Completed!")
except Exception as e:
    raise e

[2024-05-04 17:23:18,295 INFO common - yaml file: config\config.yaml loaded successfully]
[2024-05-04 17:23:18,297 INFO common - yaml file: params.yaml loaded successfully]
[2024-05-04 17:23:18,300 INFO common - yaml file: schema.yaml loaded successfully]
[2024-05-04 17:23:18,301 INFO common - created directory at: artifacts]
[2024-05-04 17:23:18,302 INFO common - created directory at: artifacts/data_ingestion]
Data Ingestion Config:
[2024-05-04 17:23:18,484 INFO 2149677558 - Downloaded artifacts/data_ingestion/data.zip of size ~ 23 KB from https://github.com//entbappy//Branching-tutorial//raw//master//winequality-data.zip with the following info: 
 Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: